In [5]:
import sys

import scipy.linalg

sys.path.append('../..')
from env.cmdp import *
from env.gridworld import *
from utils.geometric_tools import *
from visualization.mdp_vis import *
from scipy.special import kl_div
from scipy.spatial import ConvexHull
import numpy as np
import numpy.random as rn
import matplotlib.pyplot as plt
import math
from scipy.spatial.transform import Rotation
import chart_studio.plotly as py
from pathlib import Path

from chart_studio import plotly as py
import plotly.io as pio
pio.renderers.default = "browser"
from IPython.display import display, Latex

In [6]:
# fixed nu0, perturbed P
if 'fig' in globals():
    del fig
n = 2
m = 2
gamma = 0.9
nu0 = unit_vector(np.ones(n), ord=1)
P1 = np.array([[[1, 0],[1, 0]], [[1, 0],[1, 0]]])
delta = 0.1
P2 = np.array([[[0.1, 0.9],[0, 1]], [[0, 1],[0, 1]]])
P3 = np.array([[[0, 1],[1, 0]], [[0, 1],[0, 1]]])
env1 = CMDP(n, m, gamma, nu0 = nu0, P=P1)
env2 = CMDP(n, m, gamma, nu0 = nu0, P=P2)
env3 = CMDP(n, m, gamma, nu0 = nu0, P=P3)
fig, planes = plot_occ_spaces([env1, env2, env3], colors=['red', 'green', 'darkgray'], labels=[r'$\mathcal{M}_{P_0}$', r'$\mathcal{M}_{P_1}$', r'$\mathcal{M}_{P}$'])
fig.show()

In [8]:
# Define the expert's reward and temperature
rE = np.zeros((2,2))
rE[1,:] = 1.0
tau = 0.1

In [9]:
# Get reward via potential shaping
rhat = np.zeros((2,2))
rhat[1,:] = -1.0

In [10]:
# Compute optimal occupancies for the experts
tau = 0.1
vE1 = env1.soft_v_it(np.zeros(2), tau, r=rE, max_iters=1e3)
piE1 = env1.soft_v_greedy(vE1, tau, r=rE)
muE1 = env1.policy2stateactionocc(piE1)

vE2 = env2.soft_v_it(np.zeros(2), tau, r=rE, max_iters=1e3)
piE2 = env2.soft_v_greedy(vE2, tau, r=rE)
muE2 = env2.policy2stateactionocc(piE2)

vE3 = env3.soft_v_it(np.zeros(2), tau, r=rE, max_iters=1e3)
piE3 = env3.soft_v_greedy(vE3, tau, r=rE)
muE3 = env3.policy2stateactionocc(piE3)

start soft value iteration:
step:  5 , error:  0.045477386516538015
step:  10 , error:  0.02685394196415053
step:  15 , error:  0.01585698419041126
step:  20 , error:  0.0093633905945959
step:  25 , error:  0.005528988512202959
step:  30 , error:  0.003264812426570729
step:  35 , error:  0.0019278390897657927
step:  40 , error:  0.0011383697041157381
step:  45 , error:  0.0006721959265832966
step:  50 , error:  0.00039692497268828397
step:  55 , error:  0.00023438022712274353
step:  60 , error:  0.00013839918031366416
step:  65 , error:  8.172333198341875e-05
step:  70 , error:  4.825681030284912e-05
step:  75 , error:  2.8495163915787103e-05
step:  80 , error:  1.6826109340617457e-05
step:  85 , error:  9.935649304515515e-06
start soft value iteration:
step:  5 , error:  0.7015773865165382
step:  10 , error:  0.4142744309641504
step:  15 , error:  0.24462490874002185
step:  20 , error:  0.1444485623618963
step:  25 , error:  0.08529543158907593
step:  30 , error:  0.05036609939903336


In [11]:
# Compute optimal occupancies for rhat
tau = 0.1
vhat1 = env1.soft_v_it(np.zeros(2), tau, r=rhat, max_iters=1e3)
pihat1 = env1.soft_v_greedy(vhat1, tau, r=rhat)
muhat1 = env1.policy2stateactionocc(pihat1)

vhat2 = env2.soft_v_it(np.zeros(2), tau, r=rhat, max_iters=1e3)
pihat2 = env2.soft_v_greedy(vhat2, tau, r=rhat)
muhat2 = env2.policy2stateactionocc(pihat2)

vhat3 = env3.soft_v_it(np.zeros(2), tau, r=rhat, max_iters=1e3)
pihat3 = env3.soft_v_greedy(vhat3, tau, r=rhat)
muhat3 = env3.policy2stateactionocc(pihat3)

start soft value iteration:
step:  5 , error:  0.045477386516538015
step:  10 , error:  0.02685394196415053
step:  15 , error:  0.01585698419041126
step:  20 , error:  0.009363390594595955
step:  25 , error:  0.005528988512202959
step:  30 , error:  0.0032648124265707845
step:  35 , error:  0.0019278390897657927
step:  40 , error:  0.0011383697041157381
step:  45 , error:  0.0006721959265832966
step:  50 , error:  0.00039692497268817295
step:  55 , error:  0.0002343802271226325
step:  60 , error:  0.00013839918031366416
step:  65 , error:  8.172333198341875e-05
step:  70 , error:  4.825681030284912e-05
step:  75 , error:  2.8495163915787103e-05
step:  80 , error:  1.6826109340617457e-05
step:  85 , error:  9.935649304515515e-06
start soft value iteration:
step:  5 , error:  0.610622613483462
step:  10 , error:  0.3605665470358499
step:  15 , error:  0.21291094035919844
step:  20 , error:  0.1257217811727056
step:  25 , error:  0.07423745456466868
step:  30 , error:  0.0438364745458939


In [12]:
display(piE1, piE2, piE3)

array([[0.5, 0.5],
       [0.5, 0.5]])

array([[0.28241464, 0.71758536],
       [0.5       , 0.5       ]])

array([[9.99933867e-01, 6.61332391e-05],
       [5.00000000e-01, 5.00000000e-01]])

In [13]:
# Define Bregman divergence (the suboptimality of mu1 for the reward that makes mu2 optimal)
def D(env, mu1, mu2, tau=1.0):
    if np.all(mu1 >= 0) & np.all(mu1 <= 1) & np.all(mu2 >= 0) & np.all(mu2 <= 1):
        pi2 = env.occ2policy(mu2)
        pi1 = env.occ2policy(mu1)
        policy_kl = np.where(mu1 == 0, 0, np.where(mu2 == 0, float('inf'), mu1 * np.log(pi1 / pi2)))
        return tau / (1-env.gamma) * np.sum(policy_kl)
    else:
        return float('inf')

In [14]:
# Calculate sub-optimality of recovered policies
display(Latex(r'$\ell_{P_1}(r^E, \hat{\mu}_{P_1}$):'), D(env1, env1.policy2stateactionocc(pihat1), muE1, tau=tau), Latex(r'$\ell_{P_1}(\hat{r}, \mu^E_{P_1}$):'), D(env1, muE1, env1.policy2stateactionocc(pihat1), tau=tau))
display(Latex(r'$\ell_{P_2}(r^E, \hat{\mu}_{P_2}$):'), D(env2, env2.policy2stateactionocc(pihat2), muE2, tau=tau), Latex(r'$\ell_{P_2}(\hat{r}, \mu^E_{P_2}$):'), D(env2, muE2, env2.policy2stateactionocc(pihat2), tau=tau))
display(Latex(r'$\ell_{P_3}(r^E, \hat{\mu}_{P_3}$):'), D(env3, env3.policy2stateactionocc(pihat3), muE3, tau=tau), Latex(r'$\ell_{P_3}(\hat{r}, \mu^E_{P_3}$):'), D(env3, muE3, env3.policy2stateactionocc(pihat3), tau=tau))

<IPython.core.display.Latex object>

0.0

<IPython.core.display.Latex object>

0.0

<IPython.core.display.Latex object>

...

0.021272531364817236

<IPython.core.display.Latex object>

4.811919538023277

<IPython.core.display.Latex object>

4.187982472346701

In [20]:
# Compute the distance between rhat and rE in R^(SA)/U_P.
quotient_norm(rearrange(rhat-rE, 's a->(a s)'), env3.A_matrix(), is_orth=False)

1.5131928661531149